In [35]:
import pandas as pd
import numpy as np
import wfdb
import ast
import json

In [36]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = '/home/lrbutler/Desktop/ECGSignalClassifer/ptb-xl/'
sampling_rate=100

In [37]:
# load and convert annotation data
files = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
files.scp_codes = files.scp_codes.apply(lambda x: ast.literal_eval(x))

In [38]:
def less_than_100_check(row):
    codes = row['scp_codes']
    if 'NORM' in codes and codes['NORM'] < 100:
        return 1
    else:
        return 0
    
removal_mask = files.apply(less_than_100_check, axis=1).values
filtered_files = files[removal_mask == 0]
print(len(filtered_files))

19457


In [39]:
# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
print(len(agg_df))

44


In [40]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [41]:
# Apply diagnostic superclass
filtered_files['diagnostic_superclass'] = filtered_files.scp_codes.apply(aggregate_diagnostic)

/tmp/ipykernel_547119/316198874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_files['diagnostic_superclass'] = filtered_files.scp_codes.apply(aggregate_diagnostic)


In [42]:
filtered_files['diagnostic_superclass']

ecg_id
1        [NORM]
3        [NORM]
4        [NORM]
5        [NORM]
6        [NORM]
          ...  
21833    [STTC]
21834    [NORM]
21835    [STTC]
21836    [NORM]
21837    [NORM]
Name: diagnostic_superclass, Length: 19457, dtype: object

In [43]:
diagnostic_superclass = filtered_files['diagnostic_superclass'].value_counts()
classes = diagnostic_superclass.index
counts = diagnostic_superclass.values
frame = pd.DataFrame([co for co in zip(classes, counts)], columns=['class', 'counts'])


In [44]:
# Convert the index to a list and drop rows based on a condition
def convert_and_check(row):
    classes = row['class']
    count = row['counts']
    if 'NORM' in classes and len(classes) > 1:
        return 1    # Drop
    elif len(classes) == 0:
        return 1
    elif count < 20:
        return 1
    else:
        return 0

drop_indices = frame.apply(convert_and_check, axis=1).values
filtered_classes = frame[drop_indices == 0]
print(filtered_classes)


                  class  counts
0                [NORM]    7004
1                  [MI]    2532
2                [STTC]    2400
3                  [CD]    1708
4              [MI, CD]    1297
5           [HYP, STTC]     608
6            [MI, STTC]     599
7                 [HYP]     535
8            [STTC, CD]     471
10      [MI, HYP, STTC]     340
11            [HYP, CD]     300
12       [MI, STTC, CD]     223
13            [MI, HYP]     183
14          [STTC, HYP]     173
16        [MI, HYP, CD]     117
17  [CD, MI, HYP, STTC]      93
18      [CD, HYP, STTC]      89
19      [STTC, HYP, CD]      67
20      [STTC, CD, HYP]      55
21  [MI, STTC, HYP, CD]      48
23      [MI, STTC, HYP]      21


In [45]:
def all_values_match(x, y):
    if len(x) != len(y):
        return 0
    else:
        for value in x:
            if value not in y:
                return 0
            
    return 1

In [48]:
def in_class_list(row):
    try:
        for allowable_classes in filtered_classes['class'].values:
            if all_values_match(allowable_classes, row['diagnostic_superclass']):
                return 1
        
        return 0
    except:
        print(row['diagnostic_superclass'])
        print(filtered_classes['class'].values)

keep_indices = filtered_files.apply(in_class_list, axis=1)
filtered_files = filtered_files[keep_indices == 1]


In [49]:
len(filtered_files)

18878